In [1]:
import sys
sys.path.append("../")
import uniflocpy.uTools.data_workflow as data_workflow
import uniflocpy.uReservoir.MatBalance as MB_module
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT

200.0
-2.955857780762017e-11
{'fluid': <uniflocpy.uPVT.PVT_fluids.FluidStanding object at 0x000002382C9BD550>, 'p_reservoir_init_bar': 250, 't_reservoir_init_c': 80, 'p_reservoir_bar': array([227.60372612]), 't_reservoir_c': None, 'N_cum_oil_recovery_m3': 1516.5, 'N_cum_gas_recovery_m3': None, 'N_cum_wat_recovery_m3': None, 'rp_m3m3': 200.0, 'STOIIP_by_MB_m3': None, 'STOIIP_by_VOL_m3': 191797.99516426295, 'porosity_m': None, 'S_wat_connate_d': 0.25, 'S_oil_d': None, 'c_oil_1bar': None, 'c_wat_1bar': 4.35e-05, 'c_res_1bar': 7.25e-05, 'c_eff_1bar': None, 'b_oil_init_m3m3': 1.535596401942583, 'b_wat_init_m3m3': 1.0346354623557905, 'b_gas_init_m3m3': 0.003020219804915073, 'rs_init_m3m3': 200.0, 'b_oil_m3m3': 1.543981080312259, 'b_wat_m3m3': 1.034229506827085, 'b_gas_m3m3': 0.0031898215459613085, 'rs_m3m3': 200.0, 'p_drop_bar': 22.396273876100423, 'oil_recovery_perc': None, 'material_balance_m3': -2.955857780762017e-11, 'left_part_oil_m3': 2341.4473082935406, 'left_part_free_gas_m3': 0.0, '

In [2]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt


# Истощение пласта до давления насыщения

Уравнение материального баланса для данного случая будет выглядеть следующим образом

$$N_p B_o + W_p B_w = N B_{oi} (\frac {B_o - B_{oi} } {B_{oi} \Delta p }   + \frac{c_w S_{wc} + c_f}{1 - S_{wc}}) \Delta p  $$

т.е. для такого режима работа залежи свойственны следующие соотношения:

$$ m = 0 $$
$$ W_e = 0 $$
$$ R_s = R_{si} = R_p $$

Также в примере существует упрощение - отсутствие добычи воды:

$$ W_p = 0 $$

Далее с учетом того, что коэффициент сжимаемости нефти

$$ c_o = \frac {B_o - B_{oi} } {B_{oi} \Delta p } $$

И система является двухфазной:

$$ S_o = 1 - S_{wc} $$

Уравнение можно упростить до

$$N_p B_o + W_p B_w = N B_{oi} c_{eff} \Delta p  $$

В котором эффективная сжимаемость недонасыщенной системы:

$$ c_{eff} = \frac{c_o S_o +  c_w S_{wc} + c_f}{1 - S_{wc}} $$

При данных упращениях КИН 

$$ КИН = \frac {N}{N_P}$$

Может быть выражен следующим  образом:

$$ КИН = \frac {B_{oi} c_{eff} \Delta p }{B_o}$$


In [3]:

c_oil_1bar = 14.5 * 10 ** (-5)
c_wat_1bar = 4.35 * 10 ** (-5)
c_res_1bar = 7.25 * 10 ** (-5)

p_res_init_bar = 250
p_drop_bar = 103
t_res_init_c = 80
t_res_c = 80
S_wat_connate_d = 0.25
p_reservoir_bar = pbcal_bar = p_res_init_bar - p_drop_bar


In [4]:
MB_ex1  = MB_module.MatBalance()

In [5]:
MB_ex1.c_oil_1bar = c_oil_1bar
MB_ex1.c_wat_1bar = c_wat_1bar
MB_ex1.c_res_1bar = c_res_1bar

MB_ex1.t_reservoir_c = t_res_c
MB_ex1.t_reservoir_init_c = t_res_init_c
MB_ex1.p_reservoir_init_bar = p_res_init_bar
MB_ex1.S_wat_connate_d = S_wat_connate_d
MB_ex1.fluid.pbcal_bar = pbcal_bar


In [6]:
MB_ex1.calc_depletion_above_pb(p_reservoir_bar)

In [7]:
MB_ex1.oil_recovery_perc

3.922566415892517

# Поиск среднего пластового давления после эксплуатации



In [8]:
p_res_init_bar = 250
r_drainage_m = 250
porosity_d = 0.25
q_oil_surf_m3day = 50
h_eff_res_m = 8
c_system_1bar = 7.5 * 10 ** (-5)
t_end_year = 1
t_step_days = 1



In [9]:
p_res_init_bar = 400
t_res_init_c = 80
r_drainage_m = 250
porosity_d = 0.25
q_oil_surf_m3day = 50
h_eff_res_m = 8
c_system_1bar = 7.5 * 10 ** (-5)
t_end_year = 1
t_step_days = 30.33
S_wat_connate_d = 0.25

fluid = PVT.FluidStanding()
fluid.pbcal_bar = 300
fluid.calc(p_res_init_bar, t_res_init_c)


STOIIP_by_VOL_m3 = uc.pi * r_drainage_m ** 2 * h_eff_res_m * porosity_d * (1 - S_wat_connate_d) / fluid.bo_m3m3

N_cum_oil_recovery_m3 = q_oil_surf_m3day * t_step_days

MB = MB_module.MatBalance()
MB_data = data_workflow.Data()
fluid_data = data_workflow.Data()

MB.fluid = fluid
MB.rp_m3m3 = MB.fluid.rs_m3m3

MB.STOIIP_by_VOL_m3 = STOIIP_by_VOL_m3
c_wat_1bar = 4.35 * 10 ** (-5)
c_res_1bar = 7.25 * 10 ** (-5)

MB.c_wat_1bar = c_wat_1bar
MB.c_res_1bar = c_res_1bar

MB.t_reservoir_init_c = t_res_init_c
MB.p_reservoir_init_bar = p_res_init_bar
MB.S_wat_connate_d = S_wat_connate_d

for i in range(1, 30):
    N_cum_oil_recovery_m3 = q_oil_surf_m3day * t_step_days * i
    MB.calc_depletion_above_and_below_pb(N_cum_oil_recovery_m3)
    MB_data.get_data(MB)
    fluid_data.get_data(MB.fluid)


..\uniflocpy\uPVT\PVT_correlations.py:140: RuntimeWarning:

invalid value encountered in double_scalars

..\uniflocpy\uPVT\PVT_correlations.py:963: RuntimeWarning:

invalid value encountered in double_scalars

..\uniflocpy\uPVT\PVT_correlations.py:1016: RuntimeWarning:

invalid value encountered in double_scalars

..\uniflocpy\uPVT\PVT_correlations.py:1212: RuntimeWarning:

invalid value encountered in log

..\uniflocpy\uPVT\PVT_correlations.py:1238: RuntimeWarning:

invalid value encountered in log

..\uniflocpy\uPVT\PVT_correlations.py:1339: RuntimeWarning:

invalid value encountered in log

C:\Anaconda\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.



In [10]:
MB_data.print_all_names()

Номер 0 для получения параметра fluid
Номер 1 для получения параметра p_reservoir_init_bar
Номер 2 для получения параметра t_reservoir_init_c
Номер 3 для получения параметра p_reservoir_bar
Номер 4 для получения параметра t_reservoir_c
Номер 5 для получения параметра N_cum_oil_recovery_m3
Номер 6 для получения параметра N_cum_gas_recovery_m3
Номер 7 для получения параметра N_cum_wat_recovery_m3
Номер 8 для получения параметра rp_m3m3
Номер 9 для получения параметра STOIIP_by_MB_m3
Номер 10 для получения параметра STOIIP_by_VOL_m3
Номер 11 для получения параметра porosity_m
Номер 12 для получения параметра S_wat_connate_d
Номер 13 для получения параметра S_oil_d
Номер 14 для получения параметра c_oil_1bar
Номер 15 для получения параметра c_wat_1bar
Номер 16 для получения параметра c_res_1bar
Номер 17 для получения параметра c_eff_1bar
Номер 18 для получения параметра b_oil_init_m3m3
Номер 19 для получения параметра b_wat_init_m3m3
Номер 20 для получения параметра b_gas_init_m3m3
Номер 2

In [11]:
fluid_data.print_all_names()

Номер 0 для получения параметра gamma_gas
Номер 1 для получения параметра gamma_oil
Номер 2 для получения параметра gamma_wat
Номер 3 для получения параметра rsb_m3m3
Номер 4 для получения параметра gamma_gassp
Номер 5 для получения параметра y_h2s
Номер 6 для получения параметра y_co2
Номер 7 для получения параметра y_n2
Номер 8 для получения параметра s_ppm
Номер 9 для получения параметра par_wat
Номер 10 для получения параметра p_bar
Номер 11 для получения параметра t_c
Номер 12 для получения параметра pbcal_bar
Номер 13 для получения параметра tpb_C
Номер 14 для получения параметра bobcal_m3m3
Номер 15 для получения параметра muobcal_cP
Номер 16 для получения параметра _pb_bar
Номер 17 для получения параметра _bob_m3m3
Номер 18 для получения параметра _muob_cP
Номер 19 для получения параметра _mu_oil_cP
Номер 20 для получения параметра _mu_gas_cP
Номер 21 для получения параметра _mu_wat_cP
Номер 22 для получения параметра _mu_deadoil_cP
Номер 23 для получения параметра _rho_oil_kgm

In [46]:
def trace(data_x, data_y, namexy):
    tracep = go.Scattergl(
        x = data_x,
        y = data_y,
        name = namexy,
        mode = 'lines'
    )
    return tracep

def plot():
    layout = dict(title = 'Расчет материального баланса')

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')
    
def create_traces_list_by_num(data_x_values, data_y, num_y_list):
    trace_list = []
    for i in num_y_list:
        namexy = data_y.get_name(i)
        this_trace = trace(data_x_values, data_y.get_values(i), namexy)
        trace_list.append(this_trace)
    return trace_list

def connect_traces(traces1, trace2):
    connected_traces = []
    for i in traces1:
        connected_traces.append(i)
    for j in trace2:
        connected_traces.append(j)
    return connected_traces

In [52]:
numbers_MB_list = [10]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()

In [47]:
numbers_fluid_list = [16,26,3]
data_trace_fluid = create_traces_list_by_num(x_data_month, fluid_data,  numbers_fluid_list)
numbers_MB_list = [3]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = (connect_traces(data_trace_fluid, data_trace_MB))
plot()

In [49]:
numbers_MB_list = [29,30,31,32,33,34]
data_trace_MB = create_traces_list_by_num(x_data_month, MB_data,  numbers_MB_list)
data = data_trace_MB
plot()